# Типы данных

## 2. Типы данных в PostgreSQL

### ОБЗОР ТИПОВ ДАННЫХ

Особенностью хранения данных в БД является их строгая типизация, то есть точное и явное определение типов. Необходимость в типизации обусловлена тем, что компьютер по-разному обрабатывает даты, целые или дробные числа, строки.

Поэтому при создании таблицы БД обязательно указывают типы данных, которые будут сохраняться в каждом из её полей, — от этого зависит, какие значения допустимы в этих полях.



Типы данных в PostgreSQL можно разделить на несколько групп. К основным относятся:

        числовые типы — для хранения чисел (целых и дробных);
        типы даты/времени — для хранения даты, времени, часовых поясов;
        символьные типы — для хранения символов или строк;
        логический тип — для хранения значений типа «истина», «ложь».

## 3. Даты: основные типы

### TIMESTAMP

Timestamp — наиболее распространённый тип данных, так как он содержит и дату, и время, а также используется в любых логах событий, временных рядах и в большинстве системных таблиц.

Согласно стандарту ISO, значение выглядит как "2019-07-14 01:35:44.702165+00", где перечислены через точку год-месяц-день, время и часовой пояс.

Для получения текущего значения даты и времени в Postgres используются функции CURRENT_TIMESTAMP (есть в стандарте SQL) и NOW() (есть в большинстве баз данных).

Выполните запросы SELECT NOW() и SELECT CURRENT_TIMESTAMP.

Глядя на вывод, вы заметите, что время отличается на целое число часов от реального времени у вас (если вы не в Лондоне или, скажем, на Канарских островах). В Москве разница составит 3 часа.

### Timestamp with time zone

Timestamp with time zone позволяет хранить сведения о часовом поясе, что может быть удобно при анализе географически распределённых временных данных для единообразия хранения.

А ещё посмотрите ваше время в каком-нибудь часовом поясе, например, в Москве. Для этого выполните в Metabase запрос

select now() at time zone 'Europe/Moscow'

Указание at time zone позволяет переводить дату/время без часового пояса в дату/время с часовым поясом и обратно, а также пересчитывать значения времени для различных часовых поясов.

Выражение 	                                            Тип результата 	               Описание
timestamp without time zone AT TIME ZONE часовой_пояс 	timestamp with time zone 	Воспринимает время, заданное без указания       часового пояса, как время в указанном часовом поясе.
timestamp with time zone AT TIME ZONE часовой_пояс 	    timestamp without time zone 	Переводит значение timestamp с часовым поясом в другой часовой пояс, но не сохраняет информацию о нём в результате.
time with time zone AT TIME ZONE часовой_пояс 	        time with time zone 	Переводит время с часовым поясом в другой часовой пояс.

### DATE
С типом date вы уже знакомы, его реализация предельно проста. Отметим только, что тип timestamp (with/without time zone) можно легко перевести в соответствующую дату, используя синтаксис

"timestamp_column"::date

И наоборот, тип date преобразуется в timestamp (дата и 00:00:00) с помощью

"date_column"::timestamp

Для получения текущей даты можно использовать

select CURRENT_DATE

или

select now()::date

__
Предположим, у нас есть дата и время какого-то события и мы хотим посмотреть, к какой дате оно относится для Москвы и для UTC.

Используйте подзапрос

with x as 
(
select '2018-12-31 21:00:00+00'::timestamp with time zone ts
)
и выведите дату в ts в Московском часовом поясе и в поясе UTC.
Столбцы в выдаче: dt_msk (дата в московском часовом поясе), dt_utc (дата в UTC).

WITH x AS 
(
SELECT '2018-12-31 21:00:00+00'::timestamp WITH time zone ts
)
SELECT 
        (ts at time zone 'Europe/Moscow')::date dt_msk,
        (ts)::date dt_utc
FROM x

### Interval

Interval — тип данных, позволяющий хранить разницу между двумя временными метками. 

Интервалы хранят данные в трёх отдельных полях — месяцах, днях, секундах. Это сделано из-за 

## 4. Функции и операторы для работы с датами

## Функции

### Функция extract()

Функция extract() получает из значений даты/времени такие поля, как год или час.

day

Для значений timestamp это день месяца (1-31), для значений interval — число дней.

SELECT EXTRACT(DAY FROM TIMESTAMP '2001-02-16 20:38:40');
Результат: 16

SELECT EXTRACT(DAY FROM INTERVAL '40 days 1 minute');
Результат: 40


hour
Час (0-23).

SELECT EXTRACT(HOUR FROM TIMESTAMP '2001-02-16 20:38:40');
Результат: 20

month
Номер месяца, считая с января (1) до декабря (12).

SELECT EXTRACT(MONTH FROM TIMESTAMP '2001-02-16 20:38:40');
Результат: 2

year
Поле года. Учтите, что года 0 не было, и это следует иметь в виду, вычитая из годов нашей эры годы до нашей эры.

SELECT EXTRACT(YEAR FROM TIMESTAMP '2001-02-16 20:38:40');
Результат: 2001

isoyear
Год по недельному календарю ISO 8601, в который попадает дата (не применимо к интервалам).
SELECT EXTRACT(ISOYEAR FROM DATE '2006-01-01');
Результат: 2005

SELECT EXTRACT(ISOYEAR FROM DATE '2006-01-02');
Результат: 2006

week

Номер недели в году по недельному календарю ISO 8601. По определению, недели ISO 8601 начинаются с понедельника, а первая неделя года включает 4 января этого года. Другими словами, первый четверг года всегда оказывается в первой неделе этого года.

В системе нумерации недель ISO первые числа января могут относиться к 52-й или 53-й неделе предыдущего года, а последние числа декабря — к первой неделе следующего года.

Поэтому для получения согласованных результатов рекомендуется использовать поле isoyear в паре с week.

SELECT EXTRACT(WEEK FROM TIMESTAMP '2001-02-16 20:38:40');

Давайте посчитаем помесячную статистику по доставкам, используя функцию extract().

Напишите запрос, который выведет год, месяц и количество доставок.
Отсортируйте по году и по месяцу в порядке возрастания.
Столбцы в выдаче: year_n (номер года), month_n (номер месяца), qty (количество доставок).

select
    extract (year from s.ship_date) as year_n,
    extract(month from s.ship_date) as month_n,
    count(*)
from sql.shipment s
group by 1,2
order by 1,2

### Функция to_char()

Функция to_char() нужна для форматирования даты времени и интервалов в нужный текст.

to_char(timestamp[date],text) 	text 	преобразует время в текст 	to_char(current_timestamp, 'HH12:MI:SS')
to_char(interval, text) 	text 	преобразует интервал в текст 	to_char(interval '15h 2m 12s', 'HH24:MI:SS')

Любой текст, который относится к кодам форматирования, будет сохранён как есть. Чтобы оставлять какие-то подсказки и использовать обычные буквы, которые не встречаются в таблице выше, необходимо заключить текст, который нужно оставить без изменений, в двойные кавычки.

Предположим, мы хотим вывести сегодняшнюю дату в формате "Hello! Today is #название дня недели год.название месяца.день#" текстом. Для этого нужно выполнить следующий код: 

select to_char(now(),'"Hello! Today is" DAY yyyy-Mon-dd')

__
Напишите запрос, который выводит текст "Точное время x часов y минут z секунд" (текст в кавычки заключать не нужно), где x, y, z — часы, минуты и секунды соответственно, при условии, что сообщение нужно вывести для московского часового пояса.
Время введите в 24-часовом формате.
Столбцы в выдаче: msg (сообщение).

select 
to_char(now() at time zone 'Europe/Moscow', 'Точное время hh24 часов mi минут ss секунд') as mgs
__

### Функция date_trunc()

Функция date_trunc() позволяет отсечь заданное время, дату или дату со временем до нужной точности.

Например, если мы хотим округлить текущее время-дату до минут, то можно вызвать

 select date_trunc('minute',now())

Для получения разной степени точности вместо minute можно использовать следующие параметры:

        microseconds;
        milliseconds;
        second;
        minute;
        hour;
        day;
        week;

        month;
        quarter;
        year;
        decade;
        century;
        millennium.

__
Напишите запрос, который выведет дату доставки, округлённую до квартала, и общую массу доставок.
Отсортируйте по кварталу в порядке возрастания.
Столбцы в выдаче: q (начало квартала, тип date), total_weight (сумма масс доставок за квартал).

select
date_trunc('quarter', s.ship_date)::date as q,
sum(weight) as total_weight
from sql.shipment s
group by 1
order by 1 

__

### Математические операторы

К любой дате можно прибавить (и вычесть из неё) целое число X и получить другую дату, которая больше (меньше) изначальной.

Пример: select '2019-01-01'::date + 10
Результат: '2019-01-11'

При добавлении (или вычитании) целого числа к дате Postgres учитывает переходы между месяцами и годами и даёт верный ответ, соответствующий календарю. Учитываются даже високосные годы.

Пример: select '2019-01-01'::date + 500
Результат: '2020-05-15'

Аналогично можно вычесть из одной даты другую и получить расстояние в днях между этими датами. При такой операции тоже будет честная разница по календарю.

Пример:select '2019-02-10'::date - '2017-03-01'::date
Результат: 711

__
Напишите запрос, который выведет разницу между последним и первым днём доставки по каждому городу.
Отсортируйте по первому и второму столбцам.
Столбцы в выдаче: city_name (название города) и days_active (время от первой до последней доставки в днях).

select
c.city_name,
max(s.ship_date) - min(s.ship_date) as days_active
from sql.city c
join sql.shipment s on c.city_id = s.city_id
group by 1
order by 1,2

## 5. Строковые данные: основные типы

В Postgres есть три основных типа данных для работы со строками: character, character varying и text.

1
character

Cтрока фиксированной длины, дополненная пробелами.

Длина строки такого типа всегда одинакова и задаётся в скобках.

Например, в столбце character(5) всегда будет пять символов: строку большей длины туда вставить не получится, а строка меньшей длины будет дополняться ведущими пробелами. Слово "SQL" в таком столбце будет выглядеть как "  SQL".

Основной паттерн использования такого типа — универсальные справочники буквенных кодов, например код страны в стандарте ISO (RU, US, UK и т. д.).

2
character varying

Строка ограниченной переменной длины.

Например, в столбце типа character varying(5) нельзя будет хранить строку большей длины, но могут быть любые строки с меньшей длиной.

Этот тип данных повсеместно используется для хранения данных, поскольку позволяет ограничить ввод, сохраняя при этом возможность иметь строки произвольной длины.

3	
text

Cтрока неограниченной длины.

Самый удобный тип для пользователя, но самый тяжеловесный для администратора баз данных, так как в строку можно записать любой текст.

## 6. Функции и операторы для работы со строками

### Операторы

Соединение строк
Для начала познакомимся с оператором конкатенации строк — || (две вертикальные черты). Он позволяет объединять две и более строки.

Напишем запрос, который позволит подготовить простые select-запросы для всех таблиц из схемы.

select 'select * from '||t.table_schema||'.'||t.table_name||';' query
from information_schema.tables t
where table_schema = 'shipping'

В результате должно получиться пять SQL-запросов, по одному к каждой таблице из схемы shipping. 

Как мы видим, соединять можно и рукописный текст, и значения столбцов в любом произвольном порядке.

Важно! Если вы соединяете любую строку и NULL, то результатом будет NULL. Поэтому, если вы формируете какой-то текст на основе поля, в котором присутствует NULL, используйте оператор coalesce.

__
Составим текстовый шаблон сообщения о доставке по конкретному водителю для наших клиентов.

Напишите SQL-запрос, который выведет следующее сообщение для каждого водителя по форме:
Ваш заказ доставит водитель #Имя Фамилия#. Его контактный номер: #Номер#, где #Имя Фамилия# и #Номер# взяты из справочника водителей.
Если номер не указан, то выведите прочерк (-). Для номеров рекомендуем использовать coalesce.

Пример из таблицы для наглядности:
Ваш заказ доставит водитель Adel Al-Alawi. Его контактный номер: (901) 947-4433

Столбец к выдаче — msg (текст сообщения). 

SELECT 
        'Ваш заказ доставит водитель '||d.first_name||' '||d.last_name||'. '||'Его контактный номер: '||coalesce(d.phone,'-') msg
FROM shipping.driver d

## Функции

### UPPER() и LOWER()
Функции upper(your_text) и lower(your_text) переводят каждый символ вашего текста в верхний и нижний регистр соответственно.

__
Cоставим справочник названий клиентов, у которых более десяти доставок. Данные сохраним в нижнем регистре, чтобы передавать их в другие системы (например, для обзвона), которые не чувствительны к регистру.

Напишите запрос, который выводит все id названий клиентов, у которых более десяти доставок, в нижнем регистре. Отсортируйте результат по cust_id в порядке возрастания.
Столбцы в выдаче: cust_id (id клиента) и cust_name (название клиента в нижнем регистре).

select
    (c.cust_id),
    lower(c.cust_name) as cust_name
from sql.customer c
join sql.shipment s on c.cust_id = s.cust_id
group by 1,2
having count(distinct s.ship_id)>10
order by 1

### Replace()
